<a href="https://www.kaggle.com/code/lucifierx/stockpricepredictionusinglstm?scriptVersionId=90952306" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tesla-stock-price/Tesla.csv - Tesla.csv.csv


In [2]:
df = pd.read_csv("../input/tesla-stock-price/Tesla.csv - Tesla.csv.csv")
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,6/29/2010,19.000000,25.00,17.540001,23.889999,18766300,23.889999
1,6/30/2010,25.790001,30.42,23.299999,23.830000,17187100,23.830000
2,7/1/2010,25.000000,25.92,20.270000,21.959999,8218800,21.959999
3,7/2/2010,23.000000,23.10,18.709999,19.200001,5139800,19.200001
4,7/6/2010,20.000000,20.00,15.830000,16.110001,6866900,16.110001


In [3]:
df.tail()

,Date,Open,High,Low,Close,Volume,Adj Close
1687,3/13/2017,244.820007,246.850006,242.779999,246.169998,3010700,246.169998
1688,3/14/2017,246.110001,258.119995,246.020004,258.000000,7575500,258.000000
1689,3/15/2017,257.000000,261.000000,254.270004,255.729996,4816600,255.729996
1690,3/16/2017,262.399994,265.750000,259.059998,262.049988,7100400,262.049988
1691,3/17/2017,264.000000,265.329987,261.200012,261.500000,6475900,261.500000


In [4]:
df.shape

(1692, 7)

In [5]:
df1 = df.reset_index()['Close']
df1.head()

0    23.889999
1    23.830000
2    21.959999
3    19.200001
4    16.110001
Name: Close, dtype: float64

In [6]:
# import matplotlib.pyplot as plt
# plt.plot(df1)

# Lstm are sensitive so use Min Max Scaler

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
df1 = scaler.fit_transform(np.array(df1).reshape(-1,1))
df1

array([[0.02993635],
       [0.02971433],
       [0.02279455],
       ...,
       [0.88784039],
       [0.91122698],
       [0.9091918 ]])

# Spliting the data into Train_test set

In [8]:
training_size=int(len(df1)*0.65)
test_size = len(df1)-training_size
train_data, test_data = df1[0:training_size,:], df1[training_size: len(df),:1]
train_data.shape, test_data.shape

((1099, 1), (593, 1))

In [9]:
training_size, test_size

(1099, 593)

# Creating x_train, y_train , x_test and y_test

In [10]:
#Create Dataset
def create_dataset(dataset, time_step=1):
    datax, datay= [],[]
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step),0]  #i=0, 0,1,2,3....149   150
        datax.append(a)
        datay.append(dataset[i + time_step,0])
    return numpy.array(datax), numpy.array(datay)

In [11]:
import numpy
time_step = 150
x_train , y_train = create_dataset(train_data, time_step)
x_test, y_test = create_dataset(test_data, time_step)

In [12]:
x_train.shape, y_train.shape

((948, 150), (948,))

In [13]:
x_test.shape, y_test.shape

((442, 150), (442,))

# Reshape input into samples, time_steps, features which is required for LSTM

In [14]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1)

# Create the Lstm Stacked Model

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [16]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(150,1)))
model.add(LSTM(50, return_sequences = True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

2022-03-22 14:24:41.278778: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 14:24:41.367809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 14:24:41.368544: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 14:24:41.369678: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 150, 50)           10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 50)           20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, verbose=1)

Epoch 1/100


2022-03-22 14:24:44.551931: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-22 14:24:48.806684: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


15/15 [==============================] - 7s 83ms/step - loss: 0.0404 - val_loss: 0.0407
Epoch 2/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0050 - val_loss: 0.0074
Epoch 3/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0024 - val_loss: 0.0045
Epoch 4/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0019 - val_loss: 0.0039
Epoch 5/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0018 - val_loss: 0.0040
Epoch 6/100
15/15 [==============================] - 0s 25ms/step - loss: 0.0017 - val_loss: 0.0037
Epoch 7/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0016 - val_loss: 0.0036
Epoch 8/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0016 - val_loss: 0.0037
Epoch 9/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0015 - val_loss: 0.0034
Epoch 10/100
15/15 [==============================] - 0s 24ms/step - loss: 0.0015 - val_loss: 0.0031
Epoch 11/10

# Prediction and Accuracy

In [19]:
test_predict = model.predict(x_test)
train_predict = model.predict(x_train)

In [20]:
train_predict[1]

array([0.03364289], dtype=float32)

# Transform back to the orginial form

In [21]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

In [22]:
train_predict[1]

array([24.891655], dtype=float32)

In [23]:
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train, train_predict))

124.03085777435227

In [24]:
math.sqrt(mean_squared_error(y_test, test_predict))

225.51069977847024

# Plotting the Graph

In [25]:
# look_back = 150

# train_predict_plot = numpy.empty_like(df1)
# train_predict_plot[:,:] =np.nan
# train_predict_plot[look_back:len(train_predict)+look_back, :] = train_predict

# test_predict_plot = numpy.empty_like(df1)
# test_predict_plot[:,:] =np.nan
# test_predict_plot[len(train_predict)+(look_back*2)+1:(len(df1)-1),:]= test_predict

# # Plot

# plt.plot(scaler.inverse_transform(df1))
# plt.plot(train_predict_plot)
# plt.plot(test_predict_plot)
# plt.show()

In [26]:
x_input_1 = test_data[443:].reshape(1,-1)
x_input_1.shape

(1, 150)

In [27]:
# x_input.shape

In [28]:
x_input_1

array([[0.77638393, 0.77630991, 0.7689831 , 0.76761397, 0.76861304,
        0.77412668, 0.76646679, 0.7735346 , 0.76531967, 0.75917703,
        0.75558762, 0.7378626 , 0.723579  , 0.72605828, 0.68446565,
        0.67340139, 0.6920885 , 0.68794405, 0.67184723, 0.66115303,
        0.67532563, 0.66699969, 0.66833185, 0.68317048, 0.70159853,
        0.70507693, 0.69878624, 0.70093248, 0.70540996, 0.70918439,
        0.71488306, 0.70311572, 0.70481793, 0.6842066 , 0.69652898,
        0.73231198, 0.72383806, 0.71292185, 0.68531673, 0.66907192,
        0.6851317 , 0.68198638, 0.68720393, 0.68250444, 0.66870185,
        0.65926584, 0.67828597, 0.69478979, 0.67828597, 0.68194934,
        0.69182944, 0.69027527, 0.68990526, 0.69645496, 0.68150531,
        0.67321636, 0.64753548, 0.63728537, 0.6350651 , 0.6466844 ,
        0.65649053, 0.66289223, 0.64483419, 0.62740527, 0.63928357,
        0.61297362, 0.62155861, 0.62215063, 0.63965363, 0.62618413,
        0.62433392, 0.64894165, 0.65623147, 0.66

In [29]:
temp_input = list(x_input_1)
temp_input = temp_input[0].tolist()
temp_input

[0.7763839328468939,
 0.7763099060583587,
 0.7689831042005332,
 0.7676139657026136,
 0.7686130405657291,
 0.7741266764093395,
 0.7664667928574558,
 0.7735345953159735,
 0.7653196718180986,
 0.759177028446591,
 0.7555876191522773,
 0.7378626049409286,
 0.723579001953038,
 0.7260582758491545,
 0.6844656521603357,
 0.6734013948319547,
 0.6920884982652588,
 0.6879440527253683,
 0.6718472282170477,
 0.6611530308230562,
 0.6753256250816658,
 0.6669996928545099,
 0.6683318457112691,
 0.6831704849447366,
 0.7015985334725178,
 0.705076930337136,
 0.6987862370889723,
 0.7009324847972456,
 0.7054099565249792,
 0.7091843939362805,
 0.714883061597293,
 0.7031157181466791,
 0.704817930937828,
 0.6842065972548128,
 0.6965289843518322,
 0.732311983456158,
 0.7238380605589751,
 0.7129218494068359,
 0.68531673265301,
 0.6690719174746623,
 0.6851317008356079,
 0.6819863819646336,
 0.6872039254557604,
 0.6825044362694644,
 0.6687018538398584,
 0.6592658417207201,
 0.6782859676414531,
 0.6947897859195231,


# Future Prediction

In [30]:
lst_output=[]
n_steps=150
i=0
while(i<30):
    
    if(len(temp_input)>150):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input_1))
        x_input_1=x_input_1.reshape(1,-1)
        x_input_1 = x_input_1.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input_1, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input_1 = x_input_1.reshape((1, n_steps,1))
        yhat = model.predict(x_input_1, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

print(lst_output)

[0.9029025]
151
1 day input [[[0.77638393]
  [0.77630991]
  [0.7689831 ]
  [0.76761397]
  [0.76861304]
  [0.77412668]
  [0.76646679]
  [0.7735346 ]
  [0.76531967]
  [0.75917703]
  [0.75558762]
  [0.7378626 ]
  [0.723579  ]
  [0.72605828]
  [0.68446565]
  [0.67340139]
  [0.6920885 ]
  [0.68794405]
  [0.67184723]
  [0.66115303]
  [0.67532563]
  [0.66699969]
  [0.66833185]
  [0.68317048]
  [0.70159853]
  [0.70507693]
  [0.69878624]
  [0.70093248]
  [0.70540996]
  [0.70918439]
  [0.71488306]
  [0.70311572]
  [0.70481793]
  [0.6842066 ]
  [0.69652898]
  [0.73231198]
  [0.72383806]
  [0.71292185]
  [0.68531673]
  [0.66907192]
  [0.6851317 ]
  [0.68198638]
  [0.68720393]
  [0.68250444]
  [0.66870185]
  [0.65926584]
  [0.67828597]
  [0.69478979]
  [0.67828597]
  [0.68194934]
  [0.69182944]
  [0.69027527]
  [0.68990526]
  [0.69645496]
  [0.68150531]
  [0.67321636]
  [0.64753548]
  [0.63728537]
  [0.6350651 ]
  [0.6466844 ]
  [0.65649053]
  [0.66289223]
  [0.64483419]
  [0.62740527]
  [0.6392835

In [31]:
day_new = np.arange(1,151)
day_pred = np.arange(151, 181)
len(df1)

1692

In [32]:
# plt.plot(day_new, scaler.inverse_transform(df1[1542:]))
# plt.plot(day_pred, scaler.inverse_transform(lst_outputs))

In [33]:
# df2 = df1.tolist()
# df2.extend(lst_output)
# plt.plot(df2[1500:])

In [34]:
# df3 = scaler.inverse_transform(df2).tolist()
# plt.plot(df3)